<a href="https://colab.research.google.com/github/tarek-05/Gan_demo/blob/master/CO_Concentration_predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install geopandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 104.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.1/16.1 MB 89.7 MB/s eta 0:00:00


In [139]:
import pandas as pd
import geopandas as gpd

# Read the meta-data file
meta_data = pd.read_csv('/content/attributes.csv')

# Read the location data
location_data = gpd.read_file('/content/locations.geojson')

# Concatenate all the measurements files into a single data frame
years = ['2016', '2017', '2018']
measurements_data = pd.concat([pd.read_csv(f'/content/measurements-{y}.csv') for y in years])
print(location_data)

          id       lon        lat                   geometry
0   28079004 -3.712247  40.423853  POINT (-3.71225 40.42385)
1   28079008 -3.682319  40.421564  POINT (-3.68232 40.42156)
2   28079011 -3.677356  40.451475  POINT (-3.67736 40.45148)
3   28079016 -3.639233  40.440047  POINT (-3.63923 40.44005)
4   28079017 -3.713322  40.347139  POINT (-3.71332 40.34714)
5   28079018 -3.731853  40.394781  POINT (-3.73185 40.39478)
6   28079024 -3.747347  40.419356  POINT (-3.74735 40.41936)
7   28079027 -3.580031  40.476928  POINT (-3.58003 40.47693)
8   28079035 -3.703172  40.419208  POINT (-3.70317 40.41921)
9   28079036 -3.645306  40.407947  POINT (-3.64531 40.40795)
10  28079038 -3.707128  40.445544  POINT (-3.70713 40.44554)
11  28079039 -3.711542  40.478228  POINT (-3.71154 40.47823)
12  28079040 -3.651522  40.388153  POINT (-3.65152 40.38815)
13  28079047 -3.686825  40.398114  POINT (-3.68682 40.39811)
14  28079048 -3.690367  40.439897  POINT (-3.69037 40.43990)
15  28079049 -3.682583  

In [132]:
# Drop unnecessary columns from measurements_data
drop_cols = ['BEN', 'EBE', 'MXY', 'NMHC', 'NO_2', 'NOx', 'OXY', 'PM10', 'PM25', 'PXY', 'SO_2', 'TCH', 'TOL','O_3']
measurements_data.drop(columns=drop_cols, inplace=True)
measurements_data

,date,CO,station
0,2016-06-01 01:00:00,0.47,28079001
1,2016-06-01 01:00:00,0.59,28079003
2,2016-06-01 01:00:00,0.55,28079004
3,2016-06-01 01:00:00,0.36,28079039
4,2016-06-01 01:00:00,0.80,28079006
...,...,...,...
209443,2018-08-01 00:00:00,0.55,28079056
209444,2018-08-01 00:00:00,0.27,28079057
209445,2018-08-01 00:00:00,NaN,28079058
209446,2018-08-01 00:00:00,NaN,28079059


In [133]:
# Convert the date column to datetime format
measurements_data['date'] = pd.to_datetime(measurements_data['date'], format='%Y/%m/%d %H:%M:%S')
# Set the date column as the index of the data frame
measurements_data.set_index('date', inplace=True)
measurements_data

,CO,station
date,,
2016-06-01 01:00:00,0.47,28079001
2016-06-01 01:00:00,0.59,28079003
2016-06-01 01:00:00,0.55,28079004
2016-06-01 01:00:00,0.36,28079039
2016-06-01 01:00:00,0.80,28079006
...,...,...
2018-08-01 00:00:00,0.55,28079056
2018-08-01 00:00:00,0.27,28079057
2018-08-01 00:00:00,NaN,28079058


In [143]:
from shapely.geometry import Point
import requests
import json
# Find the closest station to the target location
target_point = Point(-3.6907764, 40.4279425)
location_data['distance'] = location_data.distance(target_point)
closest_station = location_data.loc[location_data['distance'].idxmin()]
station_id = closest_station['id']
closest_station

<ipython-input-143-9098b47b14bf>:6: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  location_data['distance'] = location_data.distance(target_point)


id                                            28079008
lon                                          -3.682319
lat                                          40.421564
geometry    POINT (-3.68231944444444 40.4215638888889)
distance                                      0.010593
Name: 1, dtype: object

In [145]:
from geopy.distance import geodesic
import json

# Load the GeoJSON file
with open('locations.geojson') as f:
    data = json.load(f)

# Define the target location
target_lat = 40.4279425
target_lon = -3.6907764

# Calculate the distance between the target location and each station location
distances = []
for feature in data['features']:
    lon, lat = feature['geometry']['coordinates']
    distance = geodesic((lat, lon), (target_lat, target_lon)).km
    distances.append(distance)

# Find the index of the closest station
index_of_closest_station = distances.index(min(distances))

# Get the properties of the closest station
closest_station_properties = data['features'][index_of_closest_station]['properties']
closest_station_properties

{'id': 28079008, 'lon': -3.68231944444444, 'lat': 40.4215638888889}